In [1]:
import os
import xarray as xr


info_ds = xr.open_dataset(
    os.path.join(os.getcwd(), "templates", "Data_1_GFD_lib_info.nc")
)
info_ds

<xarray.Dataset> Size: 11kB
Dimensions:         (M: 250, N: 386, Node: 20, teselas: 24, NN: 3, Ndir: 24,
                     date_ref: 6, CD_Wl_abc: 3, Wl_abc: 3)
Coordinates:
  * date_ref        (date_ref) int32 24B 2022 1 1 0 0 0
  * CD_Wl_abc       (CD_Wl_abc) float64 24B 0.00063 0.00723 0.00723
  * Wl_abc          (Wl_abc) int32 12B 0 100 100
  * M               (M) int32 1kB 1 2 3 4 5 6 7 ... 244 245 246 247 248 249 250
  * N               (N) int32 2kB 1 2 3 4 5 6 7 ... 380 381 382 383 384 385 386
  * NN              (NN) int32 12B 1 2 3
  * teselas         (teselas) int32 96B 1 2 3 4 5 6 7 8 ... 18 19 20 21 22 23 24
  * Ndir            (Ndir) float64 192B 1.0 2.0 3.0 4.0 ... 21.0 22.0 23.0 24.0
  * Node            (Node) int32 80B 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
Data variables: (12/18)
    lon_grid        (M) float64 2kB ...
    lon_grid_360    (M) float64 2kB ...
    lat_grid        (N) float64 3kB ...
    lon_node        (Node) float64 160B ...
    lat_node        (Node) float64 160B ...
    node_triangle   (teselas, NN) int32 288B ...
    ...              ...
    Wspeed          int32 4B ...
    ciudad          object 8B ...
    Eddy            int32 4B ...
    Chezy           int32 4B ...
    CD_ref          object 8B ...
    dt_forz         float64 8B ...

In [2]:
import numpy as np
from datetime import datetime, timedelta


dir_steps = len(info_ds.Ndir.values[::15])
N_Tsel = len(info_ds.teselas.values[::15])
Dir = np.arange(dir_steps)
Tes = np.arange(N_Tsel)
DDir, TTes = np.meshgrid(Dir, Tes)
DDir_flat = DDir.flatten()
TTes_flat = TTes.flatten()

date_ref = datetime(*info_ds.date_ref.values)
date_fin = date_ref + timedelta(hours=info_ds.simul_hours.values.item())

metamodel_parameters = {
    "direction": DDir_flat,
    "tesela": TTes_flat,
}

fixed_parameters = {
    "MapInterval": 3600 * info_ds.simul_dt_hours.values,
    "RefDate": date_ref.strftime("%Y%m%d"),
    "StartDateTime": date_ref.strftime("%Y%m%d%H%M%S"),
    "StopDateTime": date_fin.strftime("%Y%m%d%H%M%S"),
    "CDa": info_ds.CD_Wl_abc.values[0],
    "CDb": info_ds.CD_Wl_abc.values[1],
    "VCDa": info_ds.Wl_abc.values[0],
    "VCDb": info_ds.Wl_abc.values[1],
    "eddy": info_ds.Eddy.values,
    "chezy": info_ds.Chezy.values,
    "NetFile": "mesh_Tonga_net.nc",
    "W2Map": 1,
    "dt_forz": info_ds.dt_forz.values,
    "ds_GFD_info": info_ds,
    "wind_magnitude": 40.0,
    "simul_time": 26,
    "dir_steps": dir_steps,
    "grid_nc_file": os.path.join(
        os.getcwd(), "templates", "mesh_Tonga_net.nc"
    ),
}

In [3]:
metamodel_parameters

{'direction': array([0, 1, 0, 1]), 'tesela': array([0, 0, 1, 1])}

In [4]:
fixed_parameters

{'MapInterval': np.float64(1800.0),
 'RefDate': '20220101',
 'StartDateTime': '20220101000000',
 'StopDateTime': '20220101120000',
 'CDa': np.float64(0.00063),
 'CDb': np.float64(0.00723),
 'VCDa': np.int32(0),
 'VCDb': np.int32(100),
 'eddy': array(10, dtype=int32),
 'chezy': array(70, dtype=int32),
 'NetFile': 'mesh_Tonga_net.nc',
 'W2Map': 1,
 'dt_forz': array(0.5),
 'ds_GFD_info': <xarray.Dataset> Size: 11kB
 Dimensions:         (M: 250, N: 386, Node: 20, teselas: 24, NN: 3, Ndir: 24,
                      date_ref: 6, CD_Wl_abc: 3, Wl_abc: 3)
 Coordinates:
   * date_ref        (date_ref) int32 24B 2022 1 1 0 0 0
   * CD_Wl_abc       (CD_Wl_abc) float64 24B 0.00063 0.00723 0.00723
   * Wl_abc          (Wl_abc) int32 12B 0 100 100
   * M               (M) int32 1kB 1 2 3 4 5 6 7 ... 244 245 246 247 248 249 250
   * N               (N) int32 2kB 1 2 3 4 5 6 7 ... 380 381 382 383 384 385 386
   * NN              (NN) int32 12B 1 2 3
   * teselas         (teselas) int32 96B 1 2 3 4 5 6

In [5]:
from bluemath_tk.wrappers.delft3d.delft3d_wrapper import GreenSurgeModelWrapper


delft3d_wrapper = GreenSurgeModelWrapper(
    templates_dir=os.path.join(os.getcwd(), "templates"),
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir=os.path.join(os.getcwd(), "Etti_CASES"),
    templates_name=[
        "dimr_config.xml",
        "GreenSurge_GFDcase_wind.ext",
        "GreenSurge_GFDcase.mdu",
    ],
)

delft3d_wrapper.build_cases(
    cases_name_format=lambda ctx: f"GF_T_{ctx.get('tesela')}_D_{ctx.get('direction')}",
    num_workers=4,
)

2025-04-28 13:21:45,404 - GreenSurgeModelWrapper - WARNING - Parameter direction is not in the default_parameters
2025-04-28 13:21:45,404 - GreenSurgeModelWrapper - WARNING - Parameter tesela is not in the default_parameters


## Flow

In [ ]:
import os

output_dir = os.path.join(os.getcwd(), "Etti_CASES_Flow")

In [ ]:
metamodel_parameters = {
    "PavIni": [101300, 10100, 1010],
}
fixed_parameters = {
    "name": "FlowFM",
    "NetFile": "Santander_mesh_net.nc",
    # Friction
    "UnifFrictCoef": 0.023,
    "UnifFrictCoef1D": 0.023,
    "Vicouv": 0.1,
    "Dicouv": 0.1,
    "Vicoww": 1e-6,
    "Dicoww": 1e-6,
    # Salt Temperature
    "Rhomean": 1021,  # 1000
    "Salinity": 0,
    "InitialSalinity": 30,  # 0
    "InitialTemperature": 17,  # 6
    "Temperature": 0,
    # Wind
    "Wind_eq": 2,
    "CDa": 0.00063,
    "CDb": 0.00723,
    "CDc": 1,  #
    "VCDa": 0,
    "VCDb": 100,
    "VCDc": 1,  #
    "Rhoair": 1.2,
    # Time
    "RefDate": "20200711",
    "DtUser": 300,
    "DtMax": 30,
    "DtInit": 1,
    "StartDateTime": "202007110000",
    "StopDateTime": "202007111700",
    "ExtForceFileNew": "FlowFM_bnd.ext",
    "ObsFile": "obs.xyn",
    "HisInterval": 300,
    "MapInterval": 3600,  # 1200
    # Output parameters
    "Wrihis_balance": 0,
    "Wrihis_sourcesink": 0,
    "Wrihis_structure_gen": 1,
    "Wrihis_structure_dam": 1,
    "Wrihis_structure_pump": 1,
    "Wrihis_structure_gate": 1,
    "Wrihis_structure_weir": 1,
    "Wrihis_turbulence": 1,
    "Wrihis_wind": 0,
    "Wrihis_rain": 0,
    "Wrihis_temperature": 1,
    "Wrihis_heat_fluxes": 1,
    "Wrihis_salinity": 1,
    "Wrihis_density": 0,
    "Wrihis_waterlevel_s1": 1,
    "Wrihis_waterdepth": 0,
    "Wrihis_velocity_vector": 1,
    "Wrihis_upward_velocity_component": 0,
    "Wrihis_sediment": 0,
    "Wrihis_constituents": 0,
    "Wrimap_waterlevel_s0": 0,
    "Wrimap_waterlevel_s1": 1,
    "Wrimap_velocity_component_u0": 0,
    "Wrimap_velocity_component_u1": 1,
    "Wrimap_velocity_vector": 1,
    "Wrimap_upward_velocity_component": 0,
    "Wrimap_density_rho": 0,
    "Wrimap_horizontal_viscosity_viu": 1,
    "Wrimap_horizontal_diffusivity_diu": 1,
    "Wrimap_flow_flux_q1": 0,
    "Wrimap_spiral_flow": 1,
    "Wrimap_numlimdt": 1,
    "Wrimap_taucurrent": 1,
    "Wrimap_chezy": 1,
    "Wrimap_turbulence": 0,
    "Wrimap_wind": 0,
    "grid_nc_file": os.path.join(
        os.getcwd(), "templates", "flow", "Santander_mesh_net.nc"
    ),
}

In [ ]:
from bluemath_tk.wrappers.delft3d.delft3d_example import Delft3DfmModelWrapper


delft3d_wrapper = Delft3DfmModelWrapper(
    templates_dir=os.path.join(os.getcwd(), "templates"),
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir=output_dir,
    templates_name=["FlowFM.mdu"],
)

delft3d_wrapper.build_cases()